In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

from astropy.cosmology import FlatLambdaCDM
import GPy
import sys
sys.path.append('../code')
from photoz_kernels import *

cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=None)
DL = cosmo.luminosity_distance

In [2]:
max_redshift = 2.0
ab_filters = ['u', 'g', 'r', 'i', 'z']
numBands = len(ab_filters)
amp_arr = np.genfromtxt('filter_gaussian_coefficients_amp_'+''.join(ab_filters)+'.txt')
mu_arr = np.genfromtxt('filter_gaussian_coefficients_mu_'+''.join(ab_filters)+'.txt')
sig_arr = np.genfromtxt('filter_gaussian_coefficients_sig_'+''.join(ab_filters)+'.txt')
numCoefs = amp_arr.shape[0]

f_mod = np.load('CWW_redshiftgrid_zmax'+str(max_redshift)+''.join(ab_filters)+'.npy')
f_mod2 = np.load('CWW_redshiftgrid_zmax'+str(max_redshift)+''.join(ab_filters)+'_nolines.npy')
nz, nt, nf = f_mod.shape
redshiftGrid = np.linspace(0, max_redshift, num=nz)
f_mod_interps = np.zeros((nt, nf), dtype=object)
f_mod_interps2 = np.zeros((nt, nf), dtype=object)
for jf in range(nf):                            
    for it in range(nt):
        f_mod_interps[it, jf] = scipy.interpolate.interp1d(redshiftGrid, f_mod[:,it,jf], kind='cubic')
        f_mod_interps2[it, jf] = scipy.interpolate.interp1d(redshiftGrid, f_mod2[:,it,jf], kind='cubic')
        
lines_mu = np.array([ 3732.22, 5002.26, 1392.07, 1542.91, 2009.82, 2384.11, 2795.42, 
                     3174.70, 4858.05, 5401.32, 6292.06, 6556.74, 6719.12, 8775.69, 
                     3253.81, 3372.75, 3528.47, 3870.62, 4104.03, 4343.24, 5308.43, 
                     6853.21, 7027.87, 7134.49, 7227.85, 9065.54, 9230.14, 9527.33, 
                     9603.17, 1213.04, 3835.44, 3969.87 ])
lines_sig = np.array([ 23.11, 34.10, 6.41, 5.58, 22.81, 2.63, 11.14, 3.90, 3.68, 
                      11.83, 3.18, 4.39, 10.70, 6.77, 5.33, 2.14, 4.74, 9.18, 
                      7.03, 8.57, 4.43, 2.49, 11.83, 3.84, 3.13, 4.99, 4.52, 
                      5.01, 5.04, 8.20, 2.46, 2.93 ])
print lines_mu.size, lines_sig.size

32 32


In [3]:
numObjectsPerType = 1
bandsUsed = range(1,5) # [0, 4] #
numBandsUsed = len(bandsUsed)
typesUsed = np.arange(1,nt) #np.arange(3)#
numpoints = numObjectsPerType*typesUsed.size
#redshifts = np.random.uniform(low=0, high=max_redshift, size=numpoints)
if numObjectsPerType == 1:
    redshifts = np.ones((typesUsed.size,))
else:
    redshifts = np.outer(np.linspace(0, max_redshift, num=numObjectsPerType), np.ones(typesUsed.size)).T.flatten()
types = np.outer(np.ones(numObjectsPerType), typesUsed).T.flatten()

nd = numBandsUsed*numpoints
X = np.zeros((nd, 3))
Ytruth = np.zeros((nd, ))
Ynoise = np.zeros((nd, ))
Y = np.zeros((nd, ))
off = 0
Ynoisevarianceval = 0.05**2
for ip in range(numpoints):
    for jf in bandsUsed:
        X[off, 0] = types[ip]
        X[off, 1] = jf
        X[off, 2] = redshifts[ip]
        Ytruth[off] = f_mod_interps[types[ip], jf](redshifts[ip])
        Ynoise[off] = Ynoisevarianceval
        Y[off] = Ytruth[off] + np.sqrt(Ynoise[off]) * np.random.randn()
        off += 1

 /Users/bl/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


In [8]:
alpha_C = 1e3
alpha_L = 1e3
alpha_T = 0.9
var_T = 0.3
slope_T = 0.5

k = Photoz(amp_arr, mu_arr, sig_arr, lines_mu, lines_sig, var_T, slope_T, alpha_C, alpha_L, alpha_T)
m_full = GPy.models.GPRegression(X, Y[:,None], k)
m_full.kern.change_numlines(5)

if numObjectsPerType < 2 and numBandsUsed == 5:
    m_full.kern.var_T.constrain_bounded(1e-2, 1e5)
    m_full.kern.slope_T.constrain_bounded(-20*nt, 20*nt)
    m_full.kern.alpha_T.constrain_bounded(0.1, 2)
    m_full.kern.alpha_C.constrain_bounded(1e3, 1e4)
    m_full.kern.alpha_L.constrain_bounded(1e1, 1e2)
    m_full.likelihood.variance.fix(Ynoisevarianceval)
    m_full.optimize('bfgs', messages=True)

In [9]:
truths = {'alpha_C': alpha_C, 'alpha_L': alpha_L, 'alpha_T': alpha_T, 'var_T': var_T, 'slope_T': slope_T}
frac = 0.01
numit = 5
for nm in ['alpha_C', 'alpha_L', 'alpha_T', 'var_T', 'slope_T']:
    px = np.linspace(truths[nm]*(1-frac), truths[nm]*(1+frac), num=numit)
    py = 0*px
    for i, val in enumerate(px):
        setattr(k, nm, val)
        py[i] = np.sum( k.K(X) )
    setattr(k, nm, truths[nm]) 
    k.update_gradients_full(1.0, X, X)
    print nm, 'gradient', np.mean(np.diff(py)/np.diff(px)), getattr(k, nm).gradient[0]

alpha_C gradient 0.887032571033 0.887018595629
alpha_L gradient 3.45591115547e-07 3.45612345353e-07
alpha_T gradient 1327.57977287 1327.58100855
var_T gradient 147.190012947 147.190012947
slope_T gradient 2827.62162155 2827.62162155


In [13]:
frac = 0.02
numit = 10
for jj in range(X.shape[0]):
    #print 'X', X[jj,:]
    px = np.zeros((numit, X.shape[1]))
    py = np.zeros((numit, X.shape[1]))
    for i, val in enumerate(np.linspace((1-frac), (1+frac), num=numit)):
        for p in [0, 2]:
            Xb = 1*X[:,:]
            Xb[jj,p] = val * X[jj,p]
            px[i,p] = val * X[jj,p]
            py[i,p] = np.sum(k.K(Xb))

    #print 'px', px
    #print 'py', py
    #print 'numdiff', np.diff(py,axis=0)/np.diff(px,axis=0)
    print 'diff', np.mean(np.diff(py,axis=0)/np.diff(px,axis=0), axis=0)
    print 'grad', k.gradients_X(1.0, X)[jj,:]
    #stop


diff [ 10.94260421          nan  -4.92415365]
grad [ 10.94232888   0.           0.        ]
diff [ 14.27737762          nan  -2.36985202]
grad [ 14.2770891   0.          0.       ]
diff [ 14.20364614          nan   3.21714638]
grad [ 14.20335941   0.           0.        ]


 /Users/bl/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning:invalid value encountered in divide


diff [ 10.83245804          nan   9.26583685]
grad [ 10.83218962   0.           0.        ]
diff [ 20.39669129          nan -15.30003958]
grad [ 20.39464922   0.           0.        ]
diff [ 26.61217672          nan  -7.36335109]
grad [ 26.61007793   0.           0.        ]
diff [ 26.47474378          nan   9.99640574]
grad [ 26.47265826   0.           0.        ]
diff [ 20.19135665          nan  28.79070201]
grad [ 20.18936828   0.           0.        ]
diff [ 30.25297765          nan -32.06695586]
grad [ 30.24645701   0.           0.        ]
diff [ 39.47090122          nan -15.43264393]
grad [ 39.46430114   0.           0.        ]
diff [ 39.26705753          nan  20.95123809]
grad [ 39.26049974   0.           0.        ]
diff [ 29.94835658          nan  60.34173377]
grad [ 29.94201339   0.           0.        ]
diff [ 40.30037121          nan -55.45220937]
grad [ 40.28564331   0.           0.        ]
diff [ 52.57770602          nan -26.68710423]
grad [ 52.56300791   0.           